In [1]:
# this notebook is used to create the building model area
# Author: Younes IBNE-ETTALEB
# we will create a Character recognition model to recognize the Tifinagh characters from the dataset
# we will use the Keras library to create the model
# we will use the Tensorflow library to train the model
# we will use the OpenCV library to preprocess the images
# we will use the Numpy library to manipulate the images
# we will use the Matplotlib library to plot the images
# we will use the Pandas library to manipulate the data

# import the libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import os
import glob

In [ ]:
# first we will read the dataset with labels
dataset_dir = ""